Setup the environment with Kindred:

In [ ]:
!pip install kindred
!python -m spacy download pt_core_news_md

Load input file from my Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Here you need to configure the path where you saved the files in your Drive
%cd /content/drive/MyDrive/TCC/

Configure the parameters for the run:

In [ ]:
import kindred
import time
import sys

# use the same configuration used in the Jupyter notebook

# which Run? 1, 2 or 3
#chosen_run = 1 # one JSON per sentence
chosen_run = 2 # one JSON per document
#chosen_run = 3 # one JSON per relation

if (chosen_run == 1):
  src_folder = "run1"
  dst_folder = "run1_results"
  n_folds = 10
elif (chosen_run == 2):
  src_folder = "run2"
  dst_folder = "run2_results"
  n_folds = 8
elif (chosen_run == 3):
  src_folder = "run3"
  dst_folder = "run3_results"
  n_folds = 10

Run the task with cross-validation:

In [ ]:
default_stdout = sys.stdout

mainCorpus = kindred.load("pubannotation", src_folder)

i = 0

for trainCorpus, goldCorpus in mainCorpus.nfold_split(n_folds):

  sys.stdout = open(dst_folder + "results_fold_{}.txt".format(i), "w+")

  start_time = time.time()

  classifier = kindred.RelationClassifier(model="pt_core_news_md")

  classifier.train(trainCorpus)

  testCorpus = goldCorpus.clone()

  testCorpus.removeRelations()

  classifier.predict(testCorpus)

  kindred.evaluate(goldCorpus, testCorpus, metric='all', display=True)

  end_time = time.time()

  print("--- {} seconds ---".format(end_time - start_time))

  i += 1

  sys.stdout.close()

sys.stdout = default_stdout